# Figure 6 Analysis Notebook (PE-Aged)

This notebook generates panels 6A–6J for the PE-Aged sample, using only `Fibroblast (activated)` for 6A and fixed gene lists for subsequent panels.

## 0. Setup & Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import issparse
from scipy.ndimage import gaussian_filter1d
from scipy.spatial import KDTree
from matplotlib.colors import Normalize
from matplotlib.patches import Patch

# Directories
DATA_DIR = "./data"
RESULTS_DIR = "./results/figure6_PE_Aged"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Parameters
fibrotic_niches = ["Fibrotic Niche", "Ultraproximal", "Proximal"]
remote_niche = "Remote"
NUM_BINS = 50
min_std = 1e-4
min_fraction_target = 0.1
fib_fraction_threshold = 0.3
early_phase_bins = 50
fold_change_threshold = 2
sizes_range = (100, 500)
label_offset_x = 0.05
label_offset_y = 0.05
label_fontsize = 20

## 1. Load Data & Expression Matrix

In [ ]:
# Load AnnData
adata = sc.read_h5ad(f"{DATA_DIR}/PE-Aged.h5ad")

# Build expr_all DataFrame
if issparse(adata.X):
    expr_all = pd.DataFrame(adata.X.A, index=adata.obs_names, columns=adata.var_names)
else:
    expr_all = pd.DataFrame(adata.X, index=adata.obs_names, columns=adata.var_names)

## 2. Helper Functions

In [ ]:
def get_grouped_data(adata_subset, num_bins=NUM_BINS):
    pt = adata_subset.obs["pseudotime"]
    bins = np.linspace(pt.min(), pt.max(), num_bins+1)
    pt_bin = pd.cut(pt, bins=bins, labels=range(num_bins), include_lowest=True).astype(int)
    df = pd.DataFrame(
        adata_subset.X.A if issparse(adata_subset.X) else adata_subset.X,
        index=adata_subset.obs_names, columns=adata_subset.var_names
    )
    df["pt_bin"] = pt_bin.values
    grp = df.groupby("pt_bin").mean().reindex(range(num_bins), fill_value=0)
    centers = (bins[:-1] + bins[1:]) / 2
    return grp, centers

def filter_genes_by_conditions(adata, target_cell_type):
    tgt = adata.obs["cell_type2"] == target_cell_type
    expr_tgt = expr_all.loc[tgt]
    cond = (expr_tgt.std(axis=0) > min_std) & ((expr_tgt > 0).mean(axis=0) > min_fraction_target)
    genes = cond[cond].index.tolist()
    genes2 = []
    for g in genes:
        mask = expr_tgt[g] > 0
        frac = adata.obs.loc[tgt].loc[mask, "cell_niche"].isin(fibrotic_niches).mean()
        if frac >= fib_fraction_threshold:
            genes2.append(g)
    genes3 = []
    for g in genes2:
        ok = True
        for niche in fibrotic_niches:
            m = tgt & (adata.obs["cell_niche"] == niche)
            if m.sum()==0: ok=False; break
            grp, ctr = get_grouped_data(adata[m].copy())
            smooth = gaussian_filter1d(grp[g].values, sigma=2)
            if np.gradient(smooth, ctr)[:early_phase_bins].mean() <=0:
                ok=False; break
        if ok: genes3.append(g)
    fibm = tgt & adata.obs["cell_niche"].isin(fibrotic_niches)
    remm = tgt & (adata.obs["cell_niche"] == remote_niche)
    passed=[]
    for g in genes3:
        rem_vals = expr_all.loc[remm, g]
        if rem_vals.eq(0).all(): continue
        fc = expr_all.loc[fibm, g].mean() / (rem_vals.mean() + 1e-9)
        if fc > fold_change_threshold:
            passed.append(g)
    return passed

def calc_avg_distance(adata, genes, ct="Fibroblast (activated)"):
    tgt = adata.obs["cell_type2"] == ct
    coords_t = adata.obsm["spatial"][tgt]
    coords_f = adata.obsm["spatial"][adata.obs["cell_niche"].isin(fibrotic_niches)]
    tree = KDTree(coords_f)
    return pd.Series({g: tree.query(coords_t[expr_all.loc[tgt,g]>0], k=1)[0].mean() 
                      if (expr_all.loc[tgt,g]>0).any() else np.nan
                      for g in genes})

def calc_fib_fraction(adata, genes, ct="Fibroblast (activated)"):
    tgt = adata.obs["cell_type2"] == ct
    expr_t = expr_all.loc[tgt]
    niche = adata.obs.loc[tgt, "cell_niche"]
    return pd.Series({g: ((expr_t[g]>0)&niche.isin(fibrotic_niches)).sum()/
                      max((expr_t[g]>0).sum(),1)
                      for g in genes})

## 3. Figure 6A: Scatter for Fibroblast (activated)

In [ ]:
ct = "Fibroblast (activated)"
genes = filter_genes_by_conditions(adata, ct)
if genes:
    dist = calc_avg_distance(adata, genes, ct)
    frac = calc_fib_fraction(adata, genes, ct)
    log2fc = np.log2((expr_all.loc[adata.obs["cell_type2"]==ct,genes].mean()+1e-9)/
                     (expr_all.loc[adata.obs["cell_niche"]==remote_niche,genes].mean()+1e-9))
    df = pd.DataFrame({
        "gene": genes,
        "log2_fc": log2fc.values,
        "avg_distance": dist.values,
        "fib_fraction": frac.values
    })
    sizes = np.interp(df["fib_fraction"], (df["fib_fraction"].min(), df["fib_fraction"].max()), sizes_range)
    colors = sns.color_palette(n_colors=len(genes))
    cdict = dict(zip(genes, colors))
    fig, ax = plt.subplots(figsize=(10,8))
    for g in genes:
        row = df[df["gene"]==g]
        ax.scatter(row["log2_fc"], row["avg_distance"],
                   s=sizes[row.index], c=[cdict[g]], edgecolors='k', alpha=0.8, label=g)
        ax.text(row["log2_fc"]+label_offset_x, row["avg_distance"]+label_offset_y,
                g, fontsize=label_fontsize)
    ax.set_xlabel("log2(AvgExpr Fibrotic / Remote)")
    ax.set_ylabel("Average distance to Fibrotic Niche")
    ax.set_title(ct)
    ax.legend(title="Gene")
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/Figure6A_scatter_{ct.replace(' ','_')}.png", dpi=300)
    plt.show()

## 4. Figure 6B: Niche % Bar for Selected Genes

In [ ]:
genes_sel = ["C1qtnf3","Comp","H19","Cthrc1"]
for g in genes_sel:
    mask = expr_all[g]>0
    cnt = adata.obs.loc[mask,"cell_niche"].value_counts(normalize=True)
    df = cnt.reindex(fibrotic_niches+[remote_niche]).fillna(0)
    fig, ax = plt.subplots(figsize=(6,4))
    df.plot(kind="bar", color=["#8A2BE2","#D62728","#FFBF00","#2ca02c"], ax=ax)
    ax.set_ylabel("Proportion")
    ax.set_title(f"Figure 6B: {g} Niche %")
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/Figure6B_{g}_niche_percent.png", dpi=300)
    plt.show()

## 5. Figures 6C–6F: Pseudotime Curves & CellType2 % Bar

In [ ]:
genes_sel = ["C1qtnf3","Comp","H19","Cthrc1"]
niche_list = fibrotic_niches + ["Intermediate","Remote"]
for i, g in enumerate(genes_sel):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,5), sharey=True)
    for niche in niche_list:
        sub = adata[(adata.obs["cell_type2"]=="Fibroblast (activated)") & (adata.obs["cell_niche"]==niche)]
        if sub.n_obs:
            grp, ctr = get_grouped_data(sub)
            curve = gaussian_filter1d(grp[g].values, sigma=2)
            ax1.plot(ctr, curve, label=niche)
    ax1.set_title(f"{g} Pseudotime Curve"); ax1.set_xlabel("Pseudotime"); ax1.set_ylabel("Avg Expr")
    ax1.legend()
    expr_cells = expr_all[g]>0
    ct_counts = adata.obs.loc[expr_cells,"cell_type2"].value_counts(normalize=True).reindex(adata.obs["cell_type2"].cat.categories, fillna=0)
    ax2.barh(ct_counts.index, ct_counts.values)
    ax2.set_title(f"{g} CellType2 %"); ax2.set_xlabel("Proportion")
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/Figure6{chr(67+i)}_{g}.png", dpi=300)
    plt.show()

## 6. Figures 6G–6J: Spatial Plot Only

In [ ]:
from matplotlib.colors import Normalize
from matplotlib.patches import Patch
genes_sel = ["C1qtnf3","Comp","H19","Cthrc1"]
spatial = adata.obsm["spatial"].values
for i, g in enumerate(genes_sel):
    vals = expr_all[g].fillna(0)
    norm = Normalize(vmin=vals.min(), vmax=vals.max())
    fig, ax = plt.subplots(figsize=(6,6))
    ax.scatter(spatial[:,0], spatial[:,1], color="lightgray", s=1)
    nz = vals>0
    sc = ax.scatter(spatial[nz,0], spatial[nz,1], c=vals[nz], cmap="viridis", norm=norm, s=0.5)
    ax.set_aspect("equal"); ax.axis("off")
    ax.set_title(f"{g} Spatial")
    fig.colorbar(sc, ax=ax, label="Expression")
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/Figure6{chr(71+i)}_{g}_spatial.png", dpi=300)
    plt.show()